In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

print(model.summary())

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [ ]:
import s3fs
from PIL import Image
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

train_dir = "s3://sagemaker-dogs-vs-cats/train/"
validation_dir = "s3://sagemaker-dogs-vs-cats/validation/"
test_dir = "s3://sagemaker-dogs-vs-cats/test/"
bucket_name = "sagemaker-dogs-vs-cats"


fs = s3fs.S3FileSystem()


In [ ]:
# To List 5 files in your accessible bucket
train_list = fs.listdir(train_dir)

x_train = None
y_train = None
count = 0 
for train in train_list:
    if(train['size'] == 0):
        continue    
        
    file_name = train['name'] 
    
    with fs.open(f's3://'+file_name) as f:
        img = Image.open(f)
        resize_img = img.resize((150, 150))
        x = np.asarray(resize_img) # (150, 150, 3) 크기의 넘파이 배열
        x = x.reshape((1,) + x.shape) # (1, 150, 150, 3) 크기로 변환
        x = x.astype('float32')
        x /= 255.0
        
        y = 0
        if('cat' in file_name):
            y = 0
        elif('dog' in file_name):
            y = 1
        else:
            y = -1
        
        if(count == 0):
            x_train = x
            y_train = np.array([y])
        else:
            x_train = np.vstack((x_train,x))
            y = np.array([y])
            y_train = np.hstack((y_train,y))
        
        
    count += 1
        
#print(x_train)
print(y_train)


In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
# To List 5 files in your accessible bucket
test_list = fs.listdir(validation_dir)


x_test = None
y_test = None
count = 0 
for test in test_list:
    if(test['size'] == 0):
        continue
        
    file_name = test['name']     
    
    with fs.open(f's3://'+test['name']) as f:
        img = Image.open(f)
        resize_img = img.resize((150, 150))
        x = np.asarray(resize_img) # (150, 150, 3) 크기의 넘파이 배열
        x = x.reshape((1,) + x.shape) # (1, 150, 150, 3) 크기로 변환
        x = x.astype('float32')
        x /= 255.0
        
        y = 0
        if('cat' in file_name):
            y = 0
        elif('dog' in file_name):
            y = 1
        else:
            y = -1
        
        if(count == 0):
            x_test = x
            y_test = np.array([y])
        else:
            x_test = np.vstack((x_test,x))
            y = np.array([y])
            y_test = np.hstack((y_test,y))
        
    count += 1
        
print(test_datagen)

In [ ]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

history = model.fit_generator(
    train_datagen.flow(x_train,y_train,batch_size=20),
    steps_per_epoch=100,
    epochs=30,
    validation_data = test_datagen.flow(x_test,y_test,batch_size=20),
    validation_steps=50
)

